### Step1 导入相关包

In [1]:
import os

from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from sentence_transformers import CrossEncoder

d:\Download\miniconda3\envs\my_env\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\Download\miniconda3\envs\my_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step2 加载数据

In [2]:
data = "../PowerMarketQA/"
# 这里为了做演示，随便找了一个pdf

In [3]:
# 加载外部知识
loader = PyPDFDirectoryLoader(data)
docs_before_split = loader.load()
# 过滤目录、附录和页码
docs= [doc.page_content.replace(str(doc.metadata['page']-7),'',1) for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 
docs[0]

'第一章 电力现货市场基础   \n             \n   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现\n电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。\n上面从组织和实操两个维度对电力市场进行了描绘。  \n通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场\n泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就\n已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广\n义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种\n地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网\n才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  \n狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场\n即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源\n的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确\n定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义\n的电力市场。  \n之所以称之为狭义的电力市场，主要原因有二。  \n1）专指现代电力市场，狭义的电力市场兴起于 20世纪 80年代，是在电力作为商品\n出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创'

In [ ]:
#文本分割
# 提取所有文本
full_text=""
for doc in docs:
    full_text += doc
full_text

# 把文本按“问题-答案”分割
paragraphs = full_text.split('？ \n')

# 对分割文档进行清洗
for i in range(0,len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('电力现货市场 101 问','').replace('第一章 电力现货市场基础','').replace('第二章 国内电力现货市场建设关键问题','').replace('第三章 电力现货市场技术支持系统','').replace('\n','').replace(' ','')

# 获取问题列表
question_list = []
for i in range(0,len(paragraphs)-1):

    last_idx = paragraphs[i].rindex(str(i+1))
    ques_str = paragraphs[i][last_idx:]
    # print(ques_str)
    question_list.append(ques_str+'？')

# print(question_list)

# 获取答案列表
answer_list = []
for i in range(1,len(paragraphs)):
    if i == len(paragraphs)-1:
        answer_list.append(paragraphs[i])
    else:
        last_idx = paragraphs[i].rindex(str(i+1))
        ans_str = paragraphs[i][:last_idx]   
        answer_list.append(ans_str) 

# 将问题与答案进行组合
QA_list = []
QA_json = {}
for i in range(0,len(question_list)):
    QA_list.append(question_list[i] + '\n' + answer_list[i])
    QA_json[question_list[i]]=answer_list[i]

print(QA_list[81])
# print(QA_json)

In [5]:
print(QA_list[0])

1.什么是电力市场？电力市场与普通商品市场有哪些差异？电力市场有哪些特征？
（1）电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为政府管制或企业自律下的垄断市场形态，尤其是在二次世界大战之后，鉴于恢复经济的需要，大多数国家都采取国家垄断的模式。直至20

In [6]:
# 将分割后的文本块封装成Document类
docs_after_split = []
for i in range(0,len(QA_list)):
    qa_doc = Document(page_content=QA_list[i], metadata={"source": "..\demo\电力现货市场101问.pdf","order":i+1})
    docs_after_split.append(qa_doc)

docs_after_split[0]

Document(page_content='1.什么是电力市场？电力市场与普通商品市场有哪些差异？电力市场有哪些特征？\n（1）电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为政府管制或企业自律下的垄断市场形态，尤其是在二次世界大战之后，鉴于恢复经济

### Step3 创建向量数据库

In [10]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openai embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [11]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

### Step4 创建QA链

In [12]:
query = """输配电价对现货市场有什么影响？"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=3)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 16.什么是现货市场中的再调度法？一般用于解决什么问题？
再调度法（re-dispatch）是一种实时消除阻塞的方法，这种阻塞管理方法不直接改变日前市场中标的发电量和价格，而是一个独立、自愿参与的市场，发电机组可以在其日前发电计划的基础上，对向上或向下调整的发电量进行报价，即机组的上调、下调报价，并在实际调用后获得收益。（1）再调度法原理。系统运营商作为再调度市场的唯一买方，根据机组报价、出力约束、线路潮流分布因子等约束条件，以系统再次调节成本最低为目标，安排发电机组调整计划。具体来说，当系统发生阻塞时，在阻塞线路受端，需要对未调度发电机组按报价由低到高的顺序安排机组发电，这部分机组称为限上（constrained-on）机组，对于限上机组的多发电量按机组上调报价进行结算；在阻塞线路送端，对已经安排调度计划的发电机组按报价由高到低的顺序安排机组停发或少发电，这部分机组称为限下（constrained-off）机组，限下机组的少发电量按系统边际电价和下调报价之差进行补偿。再调度产生的阻塞成本由两部分组成：①限上机组参与再调度的上调报价和日前市场出清边际电价的差值；②限下机组回购发电量的报价与日前市场中出清边际电价的价差。阻塞成本由系统运营商直接承担，并最终通过输电费用向用户进行分摊。（2）再调度法适用性分析。再调度法适合消除偶发性、不可预测的阻塞，可用在阻塞不经常发生且造成的阻塞费用很少的电力市场中。在阻塞问题严重的电力市场中，再调度法可能会对市场短期竞价和长期投资决策带来负面影响。从短期来看，在阻塞线路送端地区，发电厂商为了确保自己在日前市场的中标量，可能会提交低于其边际成本的报价，从而在日前市场中获得更多的电量收益；而在实时平衡市场中，能以更大的价差或更多的再调度量获得补偿费用。这种报价行为会加重系统中的阻塞，造成系统再调度费用明显增加，容易产生输电阻塞导致的局部地区市场力。从长期来看，对于需要建设额外发电容量的阻塞线路受端区域，再调度机制不能给出有效的价格引导信号。在这些区域，新投建的燃气等发电机组由于边际成本较高，不能被调用，难以获得再调度费用以覆盖其成本。因此，这可能会产生相反的激励信号，引导投资建设效率低下的电厂。（3）英国电力市场再调度示例。早期的英国电力市场采用单一买方电力库Pool模式，在该模式下，NGC采用再调度的方式进行

In [13]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "BAAI/bge-reranker-base", max_length=512, device="cpu"
)
reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)
reranked_docs

[{'corpus_id': 2,
  'score': 0.99910444,
  'text': '19.输配电价的主要定价方法是什么？输配电价对现货市场有什么影响？我国输配电价的形成机制和执行方式是什么？\n（1）输配电价的主要定价方法。输配电价体系设计、定价方法的选择很大程度与电网的技术和经济特性相关，并应考虑电力改革的实际需求。由于各国国情不同，电力工业改革进程千差万别，各国输配电价制定方法各有不同。输配电网具有自然垄断特性，垄断将造成市场机制失灵，必须通过政府管制制定输配电价，这是世界各国理论和实践中达成的共识。国际上，输配电费用的计算可分为会计成本法和边际成本法两类，前者根据电网过去发生的投资成本和运行成本计算输配电费用，后者根据提供输配电引起输配电网未来投资成本的微增变化计算输配电费用。1）会计成本法。分为以下几类：a）邮票法。按整个电网输送的电量或功率平均分摊整个电网的输配电成本，与输送电能距离、流入流出节点位置无关的方法。该方法首先考虑各部分特定输电设备的成本和电网运行维护费用，形成输电总成本后，再按输电功率计算输电费。这种方法算法简单、直接透明，有利于维持电力交易的同一性和流畅性，同时也降低了独立发电厂投资的风险性。但是由于没有考虑发电资源和输电成本差异，该方法使得市场化前不同价区的发电价格趋同，使市场平均购电价格上扬，不能起到引导电源投资的经济信号作用。b）兆瓦千米法。计算输配电网所有线路和设备的每千瓦每千米的成本，通过潮流计算确定某项输配电服务的实际距离，按输送电能的实际距离和功率计算费用的方法。该方法适用于链条型输电网，对于公共输电网络难于进行合理的成本分摊和定价，也没有很好的考虑交易导致的阻塞成本以及额外的输电损失成本，实际上并不能反映每个交易对输电设备的实际使用度，从而降低了该方法的公正性。c）边界潮流法。先计算电网的简化潮流图，再根据潮流分布按各电压等级功率或电量的来源比例分摊输配电成本的方法。该方法体现了发电资源和输电成本的差异，不会由此造成市场平均购电价格上扬，对相关节点参与现货市场采取何种报价策略有一定影响，进而具有有效引导电源投资的经济信号作用。由于输配电价最终分摊至各项输配电服务，形成服务价格，因此输配电价格机制与电力现货市场交易价格形成机制密切相关。不同输配电价定价方法（如邮票法、兆瓦千米法和边界潮流法）对现货市场有不同

In [374]:
# 创建检索器
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [14]:

""" 使用自定义的检索器,重写_get_relevant_documents方法来实现对相似性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device="cpu")
        relevant_documents = vectorstore.similarity_search(query,k=10)
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)

        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(
               Document(page_content=doc["text"],
                        metadata=relevant_documents[doc["corpus_id"]].metadata)
                        ) 
        return reranking_relevant_documents

In [15]:
# 自定义加入重排的检索器
retriever = CustomRetriever()

In [16]:
# 创建prompt
prompt_template = """
请参考下面的相关文本回答问题，一段文本中可能有多个子问题，子问题以问号“？”分隔，子问题对应的答案以小括号“（）”进行分隔，
检索到与问题相关的子问题直接输出子问题对应的答案的全部内容，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

prompt = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [17]:
# 创建百川的LLM
llm = BaichuanLLM(model='Baichuan3-Turbo-128k')

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

### Step5 提问，运行QA链，得到RAG结果

In [19]:
# question = "电力市场与普通商品市场的差异？"
# question = "什么是狭义的电力市场？"
# question = "电力市场有哪些特征？"
question = "输配电价对现货市场有什么影响？"
result = qa_chain({"query": question})
# result["result"]
print("========= chain result ==========")
print(result['result'])

d:\miniconda3\envs\transformers\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


========= chain result ==========
输配电价机制是电力市场机制的重要部分，同样也是影响电力现货市场的重要因素。一般来说，电力现货市场的设计目标是理顺电力价格的形成机制，提高能源利用效率，降低电力成本，促进节能环保。电力现货市场设计应能够通过发现完整的电力价格信号，引导市场主体开展电力交易，为资源优化配置、规划投资提供可量化的参考依据，实现电力市场交易与电力系统实际运行的高效匹配。其核心是发现完整的电力价格信号。输配电价机制会影响电力现货市场设计，影响参与现货市场的市场主体的收益，进而会影响电力现货市场运行，主要表现在以下方面。1）输配电价机制影响电力现货市场设计。电价形成是电力市场的核心，涉及发电厂商、电网企业以及电力用户的利益。输配电价是电力产业价格链的中间环节，合理的输配电价机制是电力现货市场的基础和关键，它需要反映电力生产的真实成本和市场供需关系，从而为各市场成员提供明确的价格信号，正确引导市场消费和电力投资，优化社会资源的配置。2）输配电价机制影响参与现货市场的市场主体的收益。电价体系分为上网电价、输配电价和销售电价，在市场化的电力体制中，上网电价和销售电价由竞争产生，而输配电价往往由监管机构进行独立核定。但输配电价形成和分摊会影响市场主体参与现货市场的竞争方式，进而影响现货市场的市场主体的收益。3）输配电价机制会影响电力现货市场运行。合理的输配电价是电力市场有效运作的必要条件，输配电价空间会受到上网电价和销售电价的制约，如果电网合理成本的回收、合理收益的获取得不到根本保证，会严重地影响电网建设资本金的筹集。合理的输配电费用定价和分摊有利于向电力交易主体提供有效的经济信号，促进电力资源优化配置，并促进电网企业更有效地管理、规划输配电资源。


In [370]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    # print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 2 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: ..\demo\电力现货市场101问.pdf
Content: 19.输配电价的主要定价方法是什么？输配电价对现货市场有什么影响？我国输配电价的形成机制和执行方式是什么？
（1）输配电价的主要定价方法。输配电价体系设计、定价方法的选择很大程度与电网的技术和经济特性相关，并应考虑电力改革的实际需求。由于各国国情不同，电力工业改革进程千差万别，各国输配电价制定方法各有不同。输配电网具有自然垄断特性，垄断将造成市场机制失灵，必须通过政府管制制定输配电价，这是世界各国理论和实践中达成的共识。国际上，输配电费用的计算可分为会计成本法和边际成本法两类，前者根据电网过去发生的投资成本和运行成本计算输配电费用，后者根据提供输配电引起输配电网未来投资成本的微增变化计算输配电费用。1）会计成本法。分为以下几类：a）邮票法。按整个电网输送的电量或功率平均分摊整个电网的输配电成本，与输送电能距离、流入流出节点位置无关的方法。该方法首先考虑各部分特定输电设备的成本和电网运行维护费用，形成输电总成本后，再按输电功率计算输电费。这种方法算法简单、直接透明，有利于维持电力交易的同一性和流畅性，同时也降低了独立发电厂投资的风险性。但是由于没有考虑发电资源和输电成本差异，该方法使得市场化前不同价区的发电价格趋同，使市场平均购电价格上扬，不能起到引导电源投资的经济信号作用。b）兆瓦千米法。计算输配电网所有线路和设备的每千瓦每千米的成本，通过潮流计算确定某项输配电服务的实际距离，按输送电能的实际距离和功率计算费用的方法。该方法适用于链条型输电网，对于公共输电网络难于进行合理的成本分摊和定价，也没有很好的考虑交易导致的阻塞成本以及额外的输电损失成本，实际上并不能反映每个交易对输电设备的实际使用度，从而降低了该方法的公正性。c）边界潮流法。先计算电网的简化潮流图，再根据潮流分布按各电压等级功率或电量的来源比例分摊输

### Step6 开始评估

In [193]:
#result['ground_truths'] = "电力批发市场按其市场属性，可分为电力实物市场与电力金融市场。一般而言，电力实物市场与电力金融市场可以通过按产品类型和市场主体的意图两个方面加以辨识。电力实物市场建设几乎是各国各地电力市场建设的重心，建设运营中普遍接受电力监管机构的监管；电力金融市场严格意义上要接受金融监管机构的监管（见问题 3）。电力金融市场涉及能源电力衍生出的金融产品的交易行为，具有金融衍生属性，包括市场结构与相关的制度安排、市场主体、产品与交易，同时也具备其特有的供求驱动因素。电力金融市场合同通常不涉及电力实物商品的交割，取而代之的是现金的交割。电力金融市场中，一般参照金融市场期货、期权交易的基本原理进行电力期货、电力期权等电力金融衍生产品交易，其交易的对象是电力金融衍生品。"

from ragas.metrics import faithfulness, answer_relevancy, context_relevancy
from ragas.langchain.evalchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m) 
    for m in [faithfulness, answer_relevancy, context_relevancy]
}

# evaluate
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")


faithfulness_score: 1.0
answer_relevancy_score: 0.8901393929213829
context_relevancy_score: 0.14893617021276595
